In [52]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [7]:


stats_list = []
for year in range(2010, 2021):

    # NBA season we will be analyzing
    # URL page we will scraping (see image above)
    url = "https://www.basketball-reference.com/leagues/NBA_{}_advanced.html".format(year)
    
    # this is the HTML from the given URL
    html = urlopen(url)
    soup = BeautifulSoup(html)
    
    # use findALL() to get the column headers
    soup.findAll('tr', limit=2)
    # use getText()to extract the text we need into a list
    headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]
    # exclude the first column as we will not need the ranking order from Basketball Reference for the analysis
    headers = headers[1:]
    
    # avoid the first header row
    rows = soup.findAll('tr')[1:]
    player_stats = [[td.getText() for td in rows[i].findAll('td')]
        for i in range(len(rows))]
    
    
    all_stats = pd.DataFrame(player_stats, columns = headers)
    stats = all_stats.copy()


    #Add the year of the stats being produced 
    stats[["Year"]] = year
    #subset stats to relevant variables 
    stats = stats[["Player", "BPM","Year"]]

    #get duplicate values 
    player_total = all_stats[all_stats["Tm"] == "TOT"]
    #add year to duplicates to make easier to concatenate
    player_total[["Year"]] = year

    stats_without_duplicates = stats[~stats["Player"].isin(player_total["Player"])]

    final_stats = stats_without_duplicates.append(player_total[["Player", "BPM", "Year"]])


    final_stats = final_stats[final_stats.Player.notnull()]
    
    
    stats_list.append(final_stats)
    # final_stats

stats_list



    
    

/Users/dapoadegbile/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:659: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[k] = np.nan
<ipython-input-7-ee735d734a1f>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_total[["Year"]] = year
/Users/dapoadegbile/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

[                Player   BPM  Year
 0        Arron Afflalo  -0.4  2010
 1        Alexis Ajinça  -5.3  2010
 2    LaMarcus Aldridge   1.2  2010
 3        Joe Alexander  -8.3  2010
 4          Malik Allen  -5.7  2010
 ..                 ...   ...   ...
 536        Al Thornton  -3.1  2010
 541   Anthony Tolliver  -1.6  2010
 544      Alando Tucker  -4.0  2010
 557       Henry Walker  -0.5  2010
 564      Hakim Warrick  -1.3  2010
 
 [442 rows x 3 columns],
                 Player   BPM  Year
 0          Jeff Adrien  -5.3  2011
 1        Arron Afflalo   1.4  2011
 2         Maurice Ager  -0.7  2011
 6        Solomon Alabi  -8.9  2011
 7         Cole Aldrich  -4.0  2011
 ..                 ...   ...   ...
 619     Jason Williams  -1.6  2011
 625        Mo Williams  -2.4  2011
 631   Shelden Williams  -2.7  2011
 634  Terrence Williams  -6.0  2011
 639     Brandan Wright  -1.4  2011
 
 [452 rows x 3 columns],
                 Player    BPM  Year
 0          Jeff Adrien   -4.3  2012
 1      

In [9]:
stats_list[5]

,Player,BPM,Year
0,Quincy Acy,-3.1,2015
1,Jordan Adams,1.4,2015
2,Steven Adams,-1.3,2015
3,Jeff Adrien,-2.4,2015
7,Alexis Ajinça,-0.3,2015
...,...,...,...
650,Mo Williams,-0.8,2015
655,Shawne Williams,-2.0,2015
659,Nate Wolters,-5.8,2015
662,Brandan Wright,3.7,2015


In [10]:
# Create one large data frame of every player and their BPM per year
bpm = pd.concat(stats_list)

In [11]:
bpm

,Player,BPM,Year
0,Arron Afflalo,-0.4,2010
1,Alexis Ajinça,-5.3,2010
2,LaMarcus Aldridge,1.2,2010
3,Joe Alexander,-8.3,2010
4,Malik Allen,-5.7,2010
...,...,...,...
629,Dion Waiters,-1.2,2020
635,Derrick Walton,-1.0,2020
644,Paul Watson,-0.4,2020
652,Andrew Wiggins,-0.3,2020


In [18]:
nba2010_2013 = bpm[(bpm["Year"] >= 2010) & (bpm["Year"] <= 2013)]

In [19]:
nba2010_2013

,Player,BPM,Year
0,Arron Afflalo,-0.4,2010
1,Alexis Ajinça,-5.3,2010
2,LaMarcus Aldridge,1.2,2010
3,Joe Alexander,-8.3,2010
4,Malik Allen,-5.7,2010
...,...,...,...
539,Jeremy Tyler,-8.9,2013
543,Beno Udrih,-1.4,2013
548,Jarvis Varnado,-9.5,2013
563,Hakim Warrick,-4.8,2013


In [32]:
bpm_list = []

for year in range(2010, 2017):
    bpm_stats = bpm[(bpm["Year"] >= year) & (bpm["Year"] <= year+3)]
    
    bpm_list.append(bpm_stats)
    



In [36]:
# Create Dataframes for each window

bpm2010_2013 = bpm_list[0]
bpm2011_2014 = bpm_list[1]
bpm2012_2015 = bpm_list[2]
bpm2013_2016 = bpm_list[3]
bpm2014_2017 = bpm_list[4]
bpm2015_2018 = bpm_list[5]
bpm2016_2019 = bpm_list[6]
# bpm_list[0].groupby(["Player"])["BPM"].mean()

In [35]:
bpm2016_2019

,Player,BPM,Year
0,Quincy Acy,-0.1,2016
1,Jordan Adams,6.9,2016
2,Steven Adams,0.2,2016
3,Arron Afflalo,-2.9,2016
4,Alexis Ajinça,-4.0,2016
...,...,...,...
679,Jonas Valančiūnas,2.6,2019
716,Christian Wood,0.4,2019
719,Delon Wright,1.5,2019
727,Tyler Zeller,-5.0,2019


In [53]:
#Groupby player to get the average bpm for each player for each 4 season window


#change the types from object to float and string
bpm2016_2019["BPM"] = bpm2016_2019["BPM"].astype(float)
bpm2016_2019["Player"] = bpm2016_2019["Player"].astype(str)

bpm2015_2018["BPM"] = bpm2015_2018["BPM"].astype(float)
bpm2015_2018["Player"] = bpm2015_2018["Player"].astype(str)

bpm2014_2017["BPM"] = bpm2014_2017["BPM"].astype(float)
bpm2014_2017["Player"] = bpm2014_2017["Player"].astype(str)

bpm2013_2016["BPM"] = bpm2013_2016["BPM"].astype(float)
bpm2013_2016["Player"] = bpm2013_2016["Player"].astype(str)

bpm2012_2015["BPM"] = bpm2012_2015["BPM"].astype(float)
bpm2012_2015["Player"] = bpm2012_2015["Player"].astype(str)

bpm2011_2014["BPM"] = bpm2011_2014["BPM"].astype(float)
bpm2011_2014["Player"] = bpm2011_2014["Player"].astype(str)

bpm2010_2013["BPM"] = bpm2010_2013["BPM"].astype(float)
bpm2010_2013["Player"] = bpm2010_2013["Player"].astype(str)





In [58]:
#Dataframes for each 4 year window starting from the 2009 - 2010 season

bpm2016 = pd.DataFrame(bpm2016_2019.groupby(["Player"])["BPM"].mean())
bpm2015 = pd.DataFrame(bpm2015_2018.groupby(["Player"])["BPM"].mean())
bpm2014 = pd.DataFrame(bpm2014_2017.groupby(["Player"])["BPM"].mean())
bpm2013 = pd.DataFrame(bpm2013_2016.groupby(["Player"])["BPM"].mean())
bpm2012 = pd.DataFrame(bpm2012_2015.groupby(["Player"])["BPM"].mean())
bpm2011 = pd.DataFrame(bpm2011_2014.groupby(["Player"])["BPM"].mean())
bpm2010 = pd.DataFrame(bpm2010_2013.groupby(["Player"])["BPM"].mean())

In [59]:
bpm2016

,BPM
Player,
A.J. Hammons,-6.600000
Aaron Brooks,-3.366667
Aaron Gordon,0.225000
Aaron Harrison,-8.533333
Aaron Holiday,-1.600000
...,...
Zhaire Smith,-5.000000
Zhou Qi,20.950000
Álex Abrines,-2.133333


In [64]:
bpm2010.to_csv("bpm2010.csv", index= True)
bpm2011.to_csv("bpm2011.csv", index= True)
bpm2012.to_csv("bpm2012.csv", index= True)
bpm2013.to_csv("bpm2013.csv", index= True)
bpm2014.to_csv("bpm2014.csv", index= True)
bpm2015.to_csv("bpm2015.csv", index= True)
bpm2016.to_csv("bpm2016.csv", index= True)

In [65]:
bpm2010

Player
A.J. Price           -1.250000
Aaron Brooks         -1.500000
Aaron Gray           -3.275000
Acie Law             -2.500000
Adam Morrison        -4.700000
                        ...   
Zabian Dowdell       -3.600000
Zach Randolph         1.100000
Zaza Pachulia        -1.925000
Zydrunas Ilgauskas   -3.250000
Ömer Aşık            -1.366667
Name: BPM, Length: 690, dtype: float64